In [1]:
import pandas as pd
import numpy as np

In [2]:
"""df = pd.read_csv('../data/processed/master_panel.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= '2021-01-01']
df.head().to_csv('../data/processed/master_panel_head.csv', index=False)

# manipulate date column so that every 3 month is actually a year 
# jan-mar 2021 -> 1995, apr-jun 2021 -> 1996, jul-sep 2021 -> 1997, oct-dec 2021 -> 1998, etc
quarters_since_2021 = ((df['date'].dt.year - 2021) * 4) + ((df['date'].dt.month - 1) // 3)
months_in_year = (df['date'].dt.month - 1) % 12
df['date'] = pd.to_datetime('1995-01-01') + pd.to_timedelta(quarters_since_2021 * 365, unit='D') + pd.to_timedelta(months_in_year * 30, unit='D')
df.to_csv('../data/processed/master_panel_2021_onwards.csv', index=False)"""
df = pd.read_csv('../data/processed/master_panel_2021_onwards.csv')

In [3]:
from ml_research_kills_alpha.support.constants import PREDICTED_COL
# drop column where PREDICTED_COL is NaN
df = df.dropna(subset=PREDICTED_COL)

2025-10-01 14:08:03.723 | INFO     | ml_research_kills_alpha.config:<module>:11 - PROJ_ROOT path is: C:\Users\ftibe\OneDrive\Desktop\ml-research-kills-alpha


In [4]:
from ml_research_kills_alpha.modeling.rolling_trainer import RollingTrainer
from ml_research_kills_alpha.modeling.algorithms.elastic_net import ElasticNetModel
from ml_research_kills_alpha.modeling.algorithms.huber_ols import HuberRegressorModel
from ml_research_kills_alpha.modeling.algorithms.neural_networks import FFNNModel

models = [ElasticNetModel(alpha=0.0001, l1_ratio=0.9), HuberRegressorModel(alpha=0.0001, epsilon=1.5), FFNNModel(2), FFNNModel(3), FFNNModel(4), FFNNModel(5)]
trainer = RollingTrainer(models=models, data=df, end_year=2007)

2025-10-01 14:08:11,989 [INFO] RollingTrainer initialized for years 2005 to 2007 with target column: 'ret'


In [5]:
results = trainer.run()
results

2025-10-01 14:08:12,000 [INFO] Training models for test year 2005...
2025-10-01 14:08:12,002 [INFO] Year 2005: Using 102 valid features out of 212 total features.
C:\Users\ftibe\OneDrive\Desktop\ml-research-kills-alpha\ml_research_kills_alpha\modeling\rolling_trainer.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_data[self.year_col] = pd.to_datetime(features_data[self.year_col])
2025-10-01 14:08:12,121 [INFO] Year 2005: Train data from start to 1998, Validation data from 1999 to 2004, Test data for 2005.
2025-10-01 14:08:12,161 [INFO] Training model: ENET for year 2005
2025-10-01 14:08:16,913 [INFO] Evaluating model: ENET for year 2005, month 2005-07
2025-10-01 14:08:16,934 [INFO] Generated predictions for ENET
2025-10-01 14:08:16,

(        permno     date     model     y_hat
 0        10026  2005-06      ENET -0.076688
 1        58975  2005-06      ENET  0.218504
 2        59010  2005-06      ENET  0.146924
 3        59045  2005-06      ENET  0.095936
 4        59176  2005-06      ENET -0.179940
 ...        ...      ...       ...       ...
 114648   19619  2006-10  Ensemble -0.260419
 114649   19599  2006-10  Ensemble  0.179575
 114650   19616  2006-10  Ensemble  0.203629
 114651   19606  2006-10  Ensemble  0.204912
 114652   19621  2006-10  Ensemble -0.211896
 
 [114653 rows x 4 columns],
 {2005: {'ENET': np.float64(0.06511993902190005),
   'OLS-H': np.float64(0.05518062684180777),
   'FFNN2': np.float64(0.06859847395324638),
   'FFNN3': np.float64(0.07344433219805974),
   'FFNN4': np.float64(0.07453351304217773),
   'FFNN5': np.float64(0.07857221270520967),
   'Ensemble': np.float64(0.0728585398412025)},
  2006: {'ENET': np.float64(0.1180492593499001),
   'OLS-H': np.float64(0.11111439536196804),
   'FFNN2': n